### Trabalho realizado pelo grupo G18

- #### José Silva A100105
- #### Alexandra Calafate A100060

# TP2 - Problema 1: Construção de uma FSM que descreva o gerador de chaves

a. Definir e codificar, em Z3 e usando o tipo BitVec para modelar a informação, uma FSM que descreva o gerador de chaves.

b. Considere as seguintes eventuais propriedades de erro:
    1. ocorrência de um “burst”  $\,\mathsf{0}^t\,$  ($t$ zeros) que ocorre em   $\,2^t\,$ passos ou menos.
    2. ocorrência de um “burst” de tamanho $\,t\,$ que repete um “burst” anterior no mesmo output em $2^{t/2}$  passos ou menos.
Tente codificar estas propriedades e verificar se são acessíveis a partir de um estado inicial aleatoriamente gerado.

# Análise Formal da Cifra A5/1

Este notebook implementa uma análise formal da cifra A5/1 usando Z3 SMT Solver. O objetivo é verificar certas propriedades de erro que podem indicar fraquezas na cifra.

## Contexto Teórico

A cifra A5/1 é uma cifra de fluxo usada no sistema GSM para criptografia de comunicações móveis. Ela utiliza três Linear Feedback Shift Registers (LFSRs) de diferentes tamanhos que são controlados por um mecanismo de clock baseado em maioria.

### Estrutura da Cifra

- LFSR0: 19 bits
- LFSR1: 22 bits
- LFSR2: 23 bits

### Propriedades a Verificar

1. Burst de zeros: sequência de t zeros consecutivos em 2^t passos ou menos
2. Burst repetido: sequência de t bits que se repete em 2^(t/2) passos ou menos

Numa cifra segura, ambas as propriedades deveriam ser FALSE, indicando que:

- Não existem sequências previsíveis longas
- Não há repetições de padrões em períodos curtos


In [2]:
from z3 import *
import time

# Constantes do sistema
# Tamanhos dos LFSRs conforme especificação original da A5/1
SIZE0, SIZE1, SIZE2 = 19, 22, 23

# Posições dos bits de controle para o mecanismo de clock
CONTROL_BIT0, CONTROL_BIT1, CONTROL_BIT2 = 8, 10, 10

# Polinômios de feedback como variáveis simbólicas
# Isto permite uma análise genérica para qualquer configuração da cifra
S0 = BitVec('S0', SIZE0)
S1 = BitVec('S1', SIZE1)
S2 = BitVec('S2', SIZE2)

In [3]:
def declare(i):
    """Declara um estado da FSM.
    
    Args:
        i (int): Índice do estado
        
    Returns:
        dict: Estado contendo os três LFSRs como variáveis simbólicas
    """
    return {
        'lfsr0': BitVec(f'lfsr0_e{i}', SIZE0),
        'lfsr1': BitVec(f'lfsr1_e{i}', SIZE1),
        'lfsr2': BitVec(f'lfsr2_e{i}', SIZE2)
    }

def get_output(state):
    """Calcula o bit de saída da cifra para um dado estado.
    
    O output é o XOR dos bits mais significativos dos três registradores.
    
    Args:
        state (dict): Estado atual contendo os três LFSRs
        
    Returns:
        BitVec: Bit de saída (0 ou 1)
    """
    return Extract(SIZE0-1, SIZE0-1, state['lfsr0']) ^ \
           Extract(SIZE1-1, SIZE1-1, state['lfsr1']) ^ \
           Extract(SIZE2-1, SIZE2-1, state['lfsr2'])

In [4]:
def trans(curr, prox):
    """
    Vamos defenir a função de transição da FSM.

    Implementa a lógica de atualização dos registradores baseada no mecanismo de 
    clock por maioria da cifra A5/1.

    Argumentos:
        curr (dict): Estado atual
        prox (dict): Próximo estado

    Retorna:
        z3.BoolRef: Fórmula Z3 que define a transição válida
    """
    # Extrai os bits de controle
    c0 = Extract(CONTROL_BIT0, CONTROL_BIT0, curr['lfsr0'])
    c1 = Extract(CONTROL_BIT1, CONTROL_BIT1, curr['lfsr1'])
    c2 = Extract(CONTROL_BIT2, CONTROL_BIT2, curr['lfsr2'])
    
    # Calcula o bit de maioria
    c = (c0 & c1) | (c1 & c2) | (c0 & c2)
    
    # Define a transição para cada LFSR
    t0 = And(Extract(CONTROL_BIT0, CONTROL_BIT0, curr['lfsr0']) == c,
             prox['lfsr0'] == (curr['lfsr0'] << 1) ^ (curr['lfsr0'] ^ S0))
    
    t1 = And(Extract(CONTROL_BIT1, CONTROL_BIT1, curr['lfsr1']) == c,
             prox['lfsr1'] == (curr['lfsr1'] << 1) ^ (curr['lfsr1'] ^ S1))
    
    t2 = And(Extract(CONTROL_BIT2, CONTROL_BIT2, curr['lfsr2']) == c,
             prox['lfsr2'] == (curr['lfsr2'] << 1) ^ (curr['lfsr2'] ^ S2))
    
    return Or(And(t0, t1), And(t0, t2), And(t1, t2), And(t0, t1, t2))

In [6]:
def analyze_a5_1(t):
    """
    Função principal que analisa as propriedades da cifra A5/1.
    
    Verifica duas propriedades de erro:
    1. Existência de sequência de t zeros
    2. Existência de sequência de tamanho t que se repete
    
    Args:
        t (int): Tamanho do burst a ser verificado
    """
    start_time = time.time()
    
    # Calcula o número de estados necessários
    max_steps_zero = 1 << t  # 2^t passos para burst de zeros
    max_steps_repeat = 1 << (t//2)  # 2^(t/2) passos para burst repetido
    max_steps = max(max_steps_zero, max_steps_repeat)
    
    print(f"Analisando com {max_steps} estados...")
    
    # Cria estados e calcula outputs
    states = [declare(i) for i in range(max_steps)]
    outputs = [get_output(state) for state in states]
    
    # Configuração do solver
    s = Solver()
    for i in range(max_steps-1):
        s.add(trans(states[i], states[i+1]))
    
    # Verifica burst de zeros
    s.push()
    zero_conditions = [And([outputs[i+j] == 0 for j in range(t)])
                      for i in range(max_steps_zero - t + 1)]
    s.add(Or(zero_conditions))
    zero_burst = s.check() == sat
    
    if zero_burst:
        m = s.model()
        print("\nExemplo de estado que leva a burst de zeros:")
        state = states[0]
        print(f" lfsr0: {format(m[state['lfsr0']].as_long(), f'0{SIZE0}b')}")
        print(f" lfsr1: {format(m[state['lfsr1']].as_long(), f'0{SIZE1}b')}")
        print(f" lfsr2: {format(m[state['lfsr2']].as_long(), f'0{SIZE2}b')}")
        print(f" output: {m.eval(get_output(state)).as_long()}")
    s.pop()
    
    # Verifica burst repetido
    s.push()
    steps_repeat = max_steps_repeat - t + 1
    repeat_conditions = [And([outputs[i+k] == outputs[j+k] for k in range(t)])
                        for i in range(steps_repeat)
                        for j in range(i + t, steps_repeat)]
    
    if repeat_conditions:
        s.add(Or(repeat_conditions))
        repeat_burst = s.check() == sat
        
        if repeat_burst:
            m = s.model()
            print("\nExemplo de estado que leva a burst repetido:")
            state = states[0]
            print(f" lfsr0: {format(m[state['lfsr0']].as_long(), f'0{SIZE0}b')}")
            print(f" lfsr1: {format(m[state['lfsr1']].as_long(), f'0{SIZE1}b')}")
            print(f" lfsr2: {format(m[state['lfsr2']].as_long(), f'0{SIZE2}b')}")
            print(f" output: {m.eval(get_output(state)).as_long()}")
    else:
        repeat_burst = False
    s.pop()
    
    # Imprime resultados
    end_time = time.time()
    print(f"\nAnálise para estado inicial arbitrário:")
    print(f"Tempo de execução: {end_time - start_time:.2f} segundos")
    print(f"Burst de {t} zeros é acessível em 2^{t} passos ou menos: {zero_burst}")
    print(f"Burst repetido de tamanho {t} é acessível em 2^{t//2} passos ou menos: {repeat_burst}")

In [7]:
# Executa a análise para t = 8
analyze_a5_1(8)

Analisando com 256 estados...

Exemplo de estado que leva a burst de zeros:
 lfsr0: 1010001001011110111
 lfsr1: 1100011111101111010100
 lfsr2: 11011110001110011000001
 output: 1

Exemplo de estado que leva a burst repetido:
 lfsr0: 1010001001011110111
 lfsr1: 0100011111101111010100
 lfsr2: 11011110001110011000001
 output: 0

Análise para estado inicial arbitrário:
Tempo de execução: 3.87 segundos
Burst de 8 zeros é acessível em 2^8 passos ou menos: True
Burst repetido de tamanho 8 é acessível em 2^4 passos ou menos: True


## Interpretação dos Resultados

Os resultados são consistentes entre execuções porque o Z3 realiza uma verificação formal/matemática, não probabilística.

### Para t = 8:

- Burst de zeros: TRUE
  - Existe configuração que permite 8 zeros consecutivos
  - Indica previsibilidade na saída
- Burst repetido: TRUE
  - Existe sequência de 8 bits que se repete em 16 passos
  - Indica periodicidade curta

### Implicações:

- A cifra A5/1 tem fraquezas estruturais
- Pode gerar sequências previsíveis
- Tem periodicidade potencialmente curta
- Justifica sua substituição em sistemas modernos

### Nota sobre a Análise:

- A análise é genérica/arbitrária (polinômios de feedback são variáveis simbólicas)
- Resultados são determinísticos (mesmos resultados entre execuções)
- Verificação é exaustiva dentro dos limites especificados

Uma cifra forte deveria resultar em FALSE para ambas as propriedades, indicando maior resistência a análise criptográfica.
